In [ ]:
# default_exp utils

# Utils

> This module offers useful utilities.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide 
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import os
import wandb
import pandas as pd
import numpy as np
from pathlib import Path
from fastcore.xtras import globtastic

# pydicom related imports
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
from kagglerecipes.wandb_utils import log_datadir_as_artifact

### Data for testing

Code to download the sample DICOM files lives in the Data module

In [ ]:
tiny_data_path = Path('data/tiny')

## Constants

Column names from Kaggle Brain Tumor DICOM files

In [ ]:
#export
kaggle_braintumor_meta_cols = ['SpecificCharacterSet','ImageType','SOPClassUID',
             'SOPInstanceUID','AccessionNumber','Modality', 'SeriesDescription', 
             'PatientID', 'MRAcquisitionType', 'SliceThickness', 
             'EchoTime', 'NumberOfAverages', 'ImagingFrequency', 'ImagedNucleus', 
             'MagneticFieldStrength', 'SpacingBetweenSlices', 
             'EchoTrainLength', 'PercentSampling', 'PercentPhaseFieldOfView',
             'PixelBandwidth', 'TriggerWindow', 'ReconstructionDiameter', 'AcquisitionMatrix',
             'FlipAngle', 'SAR', 'PatientPosition',
             'StudyInstanceUID', 'SeriesInstanceUID', 'SeriesNumber', 'InstanceNumber',
             'ImagePositionPatient', 'ImageOrientationPatient', 'Laterality',
             'PositionReferenceIndicator', 'SliceLocation', 'InStackPositionNumber',
             'SamplesPerPixel', 'PhotometricInterpretation', 'Rows', 'Columns', 'PixelSpacing',
             'BitsAllocated', 'BitsStored', 'HighBit', 'PixelRepresentation', 'WindowCenter',
             'WindowWidth', 'RescaleIntercept', 'RescaleSlope', 'RescaleType']

## DICOM Processing

Returns the metadata of a single dicom file as a dictionary.

In [ ]:
#export
def get_dicom_metadata(path_to_dicom_file, meta_cols):
    """
    Returns the metadata of a single dicom file as a dictionary.

    Params:
        path_to_dicom_file: path to the dicom file
        meta_cols: list of metadata columns to extract
    """
    dicom_object = pydicom.dcmread(path_to_dicom_file)

    col_dict_train = dict()
    for col in meta_cols: 
        try:
            col_dict_train[col] = str(getattr(dicom_object, col))
        except AttributeError:
            col_dict_train[col] = "NaN"
    
    return col_dict_train

In [ ]:
sample_file = globtastic(tiny_data_path, file_glob='*.*dcm*')[0]
dicom_metadata = get_dicom_metadata(sample_file, kaggle_braintumor_meta_cols)
assert type(dicom_metadata) == dict

Retrieve metadata for each BraTS21ID and return as a dataframe.

In [ ]:
#export
def get_all_dicom_metadata(df, meta_cols:list, scan_types:list=['FLAIR', 'T1w', 'T1wCE', 'T2w']):
    """
    Retrieve metadata for each BraTS21ID and return as a dataframe.

    Params:
        df: dataframe with BraTS21IDs
        meta_cols: list of metadata columns to extract
        scan_types: list of strings of mdedical scan types, default: ['FLAIR', 'T1w', 'T1wCE', 'T2w']
    """
    meta_cols_dict = []
    for i in range(len(df)):
        row = df.iloc[i]
        path = Path(row.path) 
        for scan_type in scan_t:
            dicomfile = os.listdir(path / scan_type)[0]
            dicom_metadata = get_dicom_metadata(path / scan_type / dicomfile, meta_cols)
            dicom_metadata['scan_type'] = scan_type
            dicom_metadata['id'] = row.BraTS21ID
            meta_cols_dict.append(dicom_metadata)
            
    return pd.DataFrame(meta_cols_dict)

Returns the correct patient id of a dicom file.

In [ ]:
#export
def get_patient_id(patient_id):
    """
    Returns the correct patient id of a dicom file.
    
    Parameters
    ----------
    patient_id: patient id of the dicom file
    """
    if patient_id < 10:
        return '0000'+str(patient_id)
    elif patient_id >= 10 and patient_id < 100:
        return '000'+str(patient_id)
    elif patient_id >= 100 and patient_id < 1000:
        return '00'+str(patient_id)
    else:
        return '0'+str(patient_id)

In [ ]:
assert get_patient_id(1) == '00001'

Returns the MRI's plane from the dicom data.


In [ ]:
#export
def get_image_plane(data):
    '''
    Returns the MRI's plane from the dicom data.

    Params:
        data: dictionary of dicom metadata
    
    '''
    x1,y1,_,x2,y2,_ = [round(j) for j in ast.literal_eval(data.ImageOrientationPatient)]
    cords = [x1,y1,x2,y2]

    if cords == [1,0,0,0]:
        return 'coronal'
    if cords == [1,0,0,1]:
        return 'axial'
    if cords == [0,1,0,0]:
        return 'sagittal'

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_data.ipynb.
Converted 00_preprocess.ipynb.
Converted 00_utils.ipynb.
Converted index.ipynb.
